In [2]:
address = input("Please enter your Flanders address: ")
print(address)


Please enter your Flanders address: pjqstraat 7 2222 anvers
pjqstraat 7 2222 anvers


In [ ]:
""" get location of geopunt.be """

In [ ]:
GET v4/Location?q={q}&latlon={latlon}&xy={xy}&type={type}&c={c}

In [ ]:
"""
q 	

[Optional] A free form search string. May contain community names, sub-community area names, zip codes, street names, house numbers or parts thereof. A minimum of two characters is required.

	

Define this parameter in the request URI.
latlon 	

[Optional] A coordinate in WGS84 projection. Input takes the form [lat,lon] where lat and lon are provided in decimal degrees.

	

Define this parameter in the request URI.
xy 	

[Optional] A coordinate in Lambert-72 projection. Input takes the form [x,y] where x and y are provided in meters.

	

Define this parameter in the request URI.
type 	

[Optional] The location types you want to restrict the result to. Valid types are 'Municipality' (1), 'Thoroughfarename' (2), 'Housenumber' (3). The filter has no effect on reverse geocoding requests.

	

Define this parameter in the request URI.
c 	

[Optional] The maximum number of items you want to retrieve in the result. You may not ask more than 5 items. If omitted defaults to 1.

	

Define this parameter in the request URI.
"""

In [28]:
import requests
import json
import numpy as np
import pandas as pd
import utils
import utils.api

address = "Lange Van Sterbeeckstraat 8, 2060 Antwerpen"

address_info = API.get_details(address)
print(f"Address: {address}")

    

ModuleNotFoundError: No module named 'utils.api'

In [19]:
print(address)
print(get_address_details(address))

Lange Van Sterbeeckstraat 8, 2060 Antwerpen
None


In [69]:
import requests
import json
import numpy as np
import pandas as pd

from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

import rasterio as rt
from rasterio.mask import mask

%matplotlib inline
import matplotlib.pyplot as plt

import plotly.graph_objects as go

from typing import Dict

In [83]:
 def get_address_data_from_geopunt(address: str):
    data = requests.get(f"https://loc.geopunt.be/v4/Location?q={address}").json()
    print(data,"\n")
    info = {'address' : address, 
                'x_value' : data['LocationResult'][0]['Location']['X_Lambert72'],
                'y_value' : data['LocationResult'][0]['Location']['Y_Lambert72'],
                'street' : data['LocationResult'][0]['Thoroughfarename'],
                'house_number' : data['LocationResult'][0]['Housenumber'], 
                'postcode': data['LocationResult'][0]['Zipcode'], 
                'municipality' : data['LocationResult'][0]['Municipality']}
    
    detail = requests.get("https://api.basisregisters.vlaanderen.be/v1/adresmatch", 
                          params={"postcode": info['postcode'], 
                                  "straatnaam": info['street'],
                                  "huisnummer": info['house_number']}).json()
    building = requests.get(detail['adresMatches'][0]['adresseerbareObjecten'][0]['detail']).json()
    build = requests.get(building['gebouw']['detail']).json()
    info['polygon'] = [build['geometriePolygoon']['polygon']]
    return info['polygon'][0]['coordinates'][0] 


In [84]:
address = "Lange Van Sterbeeckstraat 8, 2060 Antwerpen"
#address = input("Enter the  address: ") 

In [85]:
address_coor = get_address_data_from_geopunt(address)
print(address_coor)

{'LocationResult': [{'Municipality': 'Antwerpen', 'Zipcode': '2060', 'Thoroughfarename': 'Lange Van Sterbeeckstraat', 'Housenumber': '8', 'ID': 949971, 'FormattedAddress': 'Lange Van Sterbeeckstraat 8, 2060 Antwerpen', 'Location': {'Lat_WGS84': 51.22387124286627, 'Lon_WGS84': 4.424710567489425, 'X_Lambert72': 153908.9, 'Y_Lambert72': 212660.38}, 'LocationType': 'crab_huisnummer_afgeleidVanGebouw', 'BoundingBox': {'LowerLeft': {'Lat_WGS84': 51.22387124286627, 'Lon_WGS84': 4.424710567489425, 'X_Lambert72': 153908.9, 'Y_Lambert72': 212660.38}, 'UpperRight': {'Lat_WGS84': 51.22387124286627, 'Lon_WGS84': 4.424710567489425, 'X_Lambert72': 153908.9, 'Y_Lambert72': 212660.38}}}]} 

[[153913.09697395563, 212652.07560559362], [153909.20398195833, 212669.40699760243], [153904.5820299536, 212668.39899760485], [153907.5011979565, 212656.54043759406], [153908.57345395535, 212652.1845335923], [153908.816653952, 212651.1965655908], [153909.12673395127, 212651.2602455914], [153913.09697395563, 212652.0

In [55]:
def get_polygon(address_info: Dict[str,str]) -> Dict[str,str]:
    req = requests.get("https://api.basisregisters.vlaanderen.be/v1/adresmatch",
                       params={"postcode": address_info['postcode'],
                                "straatnaam": address_info['street'],
                                "huisnummer": address_info['house_number']}).json()
    detail = requests.get(req['adresMatches'][0]['adresseerbareObjecten'][0]['detail']).json()
    build = requests.get(detail['gebouw']['detail']).json()
    polygon = build['geometriePolygoon']['polygon']
     
    return polygon

In [56]:
get_polygon(address_coor)

TypeError: list indices must be integers or slices, not str